In [ ]:
# INITIALIZATION
# for initializing the model parameters in training
from torch.nn.init import xavier_normal # alternative initializations can be found at torch.nn.init
# CONFIGURATIONS
import config
# UTILITIES
from utils.dataset import CNNDMdataset
from utils.masks import create_mask
from utils.transforms import construct_token_transform, construct_vocab_transform, construct_tensor_transform, construct_text_transform
# DECODERS
from decoders.greedy import greedy_decode
from decoders.beamsearch import beam_search_decode
from decoders.temperature import temperature_decode
from decoders.mbrd import mbr_decode, mbr_decode_verbose, candidates_set
from decoders.uniform import uniform_decode
# TRANSFORMER MODEL
from EncoderDecoderTransformer import EncoderDecoderTransformer
# EVALUATION
from evaluation.rouge_evaluator import RougeEvaluator


# PYTORCH
import torch
# TRACKING PROGRESS
from timeit import default_timer as timer
import tqdm
import time
# DATA MANAGEMENT
import numpy as np
import pandas as pd

In [ ]:
import abstractive_summarizer

In [ ]:
import json

# CNNDM Datasets
eval_PATH = 'data/test.json'
train_PATH = 'data/train.json'
val_PATH = 'data/validation.json'

# trim the data for quick checks on laptop
e_trim = 10 # 10000
t_trim = 50 # 10000
v_trim = 20 # 10000

with open(eval_PATH, 'r') as f:
  eval_data = json.load(f)[:e_trim]
with open(train_PATH, 'r') as f:
  train_data = json.load(f)[:t_trim]
with open(val_PATH, 'r') as f:
  val_data = json.load(f)[:v_trim]

train_articles = [article['article'] for article in train_data]
train_summaries = [article['summary'] for article in train_data]
val_articles = [article['article'] for article in val_data]
val_summaries = [article['summary'] for article in val_data]
eval_articles = [article['article'] for article in eval_data]
eval_summaries = [article['summary'] for article in eval_data]

# Remove the outliers from the Training Data
# t_remove = [1469,1814]
# len(train_articles[1469]), len(train_summaries[1469])
# len(train_articles[1814]), len(train_summaries[1814])
# del train_articles[1469]
# del train_summaries[1469]
# del train_articles[1814]
# del train_summaries[1814]

In [ ]:
summarizer = abstractive_summarizer.AbstractiveSummarizer()
summarizer.train(train_articles, train_summaries, val_articles, val_summaries)

In [ ]:
temperature_decode_T = lambda T: lambda summarizer_in, src_in, src_mask_in, max_len_in, start_symbol_in: temperature_decode(summarizer_in, src_in, src_mask_in, max_len_in, start_symbol_in, temperature=T)
evaluator = RougeEvaluator()
decoders = {'Random (Uniform)': uniform_decode, 
            'Temperature 0.25': temperature_decode_T(0.25),
            'Temperature 0.5': temperature_decode_T(0.5),
            'Temperature 0.75': temperature_decode_T(0.75),
            'Temperature 1 (Unbiased)': temperature_decode_T(1),
            'Temperature 1.25': temperature_decode_T(1.25),
            'Temperature 1.5': temperature_decode_T(1.5),
            'Temperature 1.75': temperature_decode_T(1.75),
            'Temperature 2': temperature_decode_T(2),
            'Temperature 5': temperature_decode_T(5),
            'Temperature 10': temperature_decode_T(10),
            'Beam Search' : beam_search_decode,
            'Greedy': greedy_decode,
            'MBR' : mbr_decode
            }
pred_summaries_dict = {}

In [ ]:
decoder_name = 'Greedy'
predictor = summarizer.predict(eval_articles, decoder=decoders[decoder_name])
pred_summaries = [s for s in predictor]
eval_articles = [article['article'] for article in eval_data]
eval_out_data = [{'article': article, 'summary': summary} for article, summary in zip(eval_articles, pred_summaries)]
pred_summaries_dict[decoder_name] = eval_out_data

In [ ]:
decoder_name = 'Temperature 0.5'
predictor = summarizer.predict(eval_articles, decoder=decoders[decoder_name])
pred_summaries = [s for s in predictor]
eval_articles = [article['article'] for article in eval_data]
eval_out_data = [{'article': article, 'summary': summary} for article, summary in zip(eval_articles, pred_summaries)]
pred_summaries_dict[decoder_name] = eval_out_data

In [ ]:
decoder_name = 'Temperature 1 (Unbiased)'
predictor = summarizer.predict(eval_articles, decoder=decoders[decoder_name])
pred_summaries = [s for s in predictor]
eval_articles = [article['article'] for article in eval_data]
eval_out_data = [{'article': article, 'summary': summary} for article, summary in zip(eval_articles, pred_summaries)]
pred_summaries_dict[decoder_name] = eval_out_data


In [ ]:
pred_summaries_dict

In [ ]:
# Write nested JSON data to a file
with open('data_pred/pred.json', 'w') as f:
    json.dump(pred_summaries_dict, f)

In [ ]:
import matplotlib.pyplot as plt

# Sample data
epochs = range(len(summarizer.train_losses))
train_losses = summarizer.train_losses
val_losses = summarizer.val_losses

# Improved plot
plt.figure(figsize=(10, 6))  # Larger figure size
plt.plot(epochs, train_losses, label='Training Loss', color='blue', linewidth=2, marker='o')
plt.plot(epochs, val_losses, label='Validation Loss', color='red', linewidth=2, marker='x')

plt.title('Cross-Entropy Loss across Epochs', fontsize=14)
plt.xlabel('Epochs', fontsize=12)
plt.ylabel('Cross-Entropy Loss', fontsize=12)
plt.legend(loc="upper right")
plt.grid(True)  # Add grid for better readability
plt.tight_layout()  # Adjusts the plot to fit into the figure area.

plt.show()

In [ ]:
summarizer.model_paths